# import

In [1]:
import pandas as pd
import json
import requests
from pathlib import Path
import numpy as np
import sys
import requests
import json
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context('talk')
%matplotlib inline


# directories


In [2]:
if sys.platform =='linux':
    path_d = Path('/mnt/d')
else:
    path_d = Path('D://')
data_dir = path_d/'data'/'dpe_full'   

plot_dir = Path('.')/'plot'
plot_dir.mkdir(exist_ok=True,parents=True)

# read exemple

In [34]:
dept = '38'

td007 =pd.read_csv(data_dir/dept/'td007_paroi_opaque.csv',dtype=str)
td006 =pd.read_csv(data_dir/dept/'td006_batiment.csv',dtype=str)
td001 =pd.read_csv(data_dir/dept/'td001_dpe.csv',dtype=str)


In [4]:
from assets_orm import DPEMetaData

In [5]:
meta = DPEMetaData()
self=meta

In [ ]:
table =td007.copy()

In [6]:
table =td007.copy()

In [7]:
table = meta.merge_all_tr_table(table)

In [8]:
table = meta.merge_all_tv_table(table)

In [ ]:
td007_types = {}

In [16]:
table = table.astype(td007_types)

In [29]:
table['surface_paroi_opaque']=table.deperdition_thermique/(table.coefficient_transmission_thermique_paroi*table.tv001_valeur.astype(float).fillna(1))


In [32]:
table.id

0             1048
1             1049
2             1050
3             1814
4             1815
            ...   
439933    24115581
439934    24115582
439935    24115588
439936    24115589
439937    24115590
Name: id, Length: 439938, dtype: object

In [31]:
table[['coefficient_transmission_thermique_paroi','surface_paroi_opaque','surface_paroi','deperdition_thermique','tv001_valeur']]


,coefficient_transmission_thermique_paroi,surface_paroi_opaque,surface_paroi,deperdition_thermique,tv001_valeur
0,0.35,115.057143,115.06,40.27,1.0
1,0.35,10.257143,10.80,3.59,NaN
2,0.14,46.357143,49.10,6.49,NaN
3,0.38,83.447368,82.67,31.71,1.0
4,0.32,12.156250,12.80,3.89,NaN
...,...,...,...,...,...
439933,0.25,69.400000,69.40,17.35,1.0
439934,0.40,114.850000,114.86,45.94,1.0
439935,0.40,14.000000,17.50,5.60,NaN
439936,0.23,40.565217,40.55,9.33,1.0
